# Packages

In [2]:
import pickle as pkl
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

import rasterio

In [ ]:
x = np.array([3,4,5])
y = np.array([[5,4,6],[1,3,2]])

In [ ]:
y


In [ ]:
dist = (y - x)**2
dist = np.sum(dist,axis = 1)
dist = np.sqrt(dist)
dist

In [ ]:
import numpy as np
single_point = [3, 4]
points = np.arange(20).reshape((10,2))

dist = (points - single_point)**2
dist = np.sum(dist, axis=1)
dist = np.sqrt(dist)
dist

In [ ]:
sum(x - y)

# Load Data
We'll focus on the 2010s for now. Maybe change to longer time period after this.

In [ ]:
compustat = pd.read_csv("../../data/companyData/compustatChanges_All.csv").drop(columns = ['Unnamed: 0'])
compustat.head()

compustat2 = pd.read_csv("../../data/companyData/compustatChanges_2010s.csv").drop(columns = ['Unnamed: 0'])
compustat2.head()

compustat = compustat.append(compustat2)

In [ ]:
compustat.year.max()

In [ ]:
compustat = compustat[(~compustat.qtr.isna()) & (compustat.year > 1996)]

In [ ]:
compustat.qtr

In [ ]:
compustat['fquarter'] = "q" + compustat.qtr.astype(int).astype(str) + "y" + compustat.year.astype(int).astype(str)
compustat.rename(columns = {'tic': 'TICKER'}, inplace = True)
compustat.reset_index(drop = True,inplace = True)

In [ ]:
compustat.revenueChangeAbsolute.sum()*0.005

In [ ]:
compustat

In [ ]:
hq = pd.read_csv("../../data/allIGData_byQuarter.csv").drop(columns = ['Unnamed: 0'])
hq

In [ ]:
compustatWithHQ = compustat.merge(hq)

In [ ]:
compustatWithHQ.revenueChangeAbsolute.sum()*0.005

In [ ]:
compustatWithHQ.head()

In [ ]:
compustatWithHQ.to_csv("../../data/compustatWithHQ_all.csv")

# Reshape the Compustat HQ Data

In [ ]:
def melt(start):    
    start = start.melt(id_vars = ['year','TICKER','company','state','latitude','longitude'],
               var_name = 'origVar',
               value_name = start.columns[7].split('_')[1])
    start['day'] = start.origVar.str[-6:-4]
    start['month'] = start.origVar.str[-8:-6]
    start['weatherYear'] = start.origVar.str[-12:-8].astype('int64')

    # print(start.shape,start.weatherYear.value_counts())

    start = start[start.year == start.weatherYear]
    start = start.drop(['origVar'],axis = 1)

    start = start.reset_index(drop=True)

    
    return(start)

In [ ]:
def getBoth(year):
    tmax = pd.read_csv("../../../../../../../Volumes/backup2/dissData/prism/hqInfogroupTmax" + str(year) + ".csv").drop(['Unnamed: 0'],axis = 1)
    precip = pd.read_csv("../../../../../../../Volumes/backup2/dissData/prism/hqInfogroupPrecip" + str(year) + ".csv").drop(['Unnamed: 0'],axis = 1)

    df1 = melt(precip)
    df2 = melt(tmax)


    allDF = df1.merge(df2)
    
    
    filename = '../../../../../../../Volumes/backup2/dissData/prism/allWeather' + str(year) + '.pkl'
    with open(filename, 'wb') as handle:
        pickle.dump(allDF, handle)
    
    # return(allDF)

In [ ]:
for year in range(2019,2020):
    print(year)
    startTime = time.time()

    getBoth(year)

    print(time.time() - startTime)

In [ ]:
allData = pd.DataFrame()
for year in range(1998,2020):
    print(year)
    callLocation = '../../../../../../../Volumes/backup2/dissData/prism/allWeather' + str(year) + '.pkl'
    
    file   = open(callLocation,"rb")

    allWeather = pkl.load(file)
    
    allData = allData.append(allWeather)
    
    

In [ ]:
allData.head()

In [ ]:
allData.to_csv("../../../../../../../Volumes/backup2/dissData/prism/allWeather.csv")

In [ ]:
allData.shape

# Merge in SC Information

In [3]:
compustatSC = pd.read_csv("../../data/companyData/compustatSCLinked.csv")# [['stic','cgvkey','cconm','srcdate']]
compustatSC.head()

,gvkey,conm,cgvkey,cconm,cnms,srcdate,cid,sid,ctype,salecs,scusip,stic,ccusip,ctic
0,1005,A.B.A. INDUSTRIES INC,10983,UNITED TECHNOLOGIES CORP,UTD TECHS,19781031,101,1,COMPANY,4.100,000370106,ABA.2,75513E101,RTX
1,1005,A.B.A. INDUSTRIES INC,10983,UNITED TECHNOLOGIES CORP,UTD TECHS,19791031,101,1,COMPANY,7.454,000370106,ABA.2,75513E101,RTX
2,1005,A.B.A. INDUSTRIES INC,8972,RAYTHEON CO,RAYTHEON,19791031,201,2,COMPANY,2.377,000370106,ABA.2,755111507,RTN
3,1005,A.B.A. INDUSTRIES INC,8972,RAYTHEON CO,RAYTHEON,19801031,201,2,COMPANY,2.292,000370106,ABA.2,755111507,RTN
4,1005,A.B.A. INDUSTRIES INC,10983,UNITED TECHNOLOGIES CORP,UTD TECHS,19801031,101,1,COMPANY,9.563,000370106,ABA.2,75513E101,RTX


In [ ]:
compustatSC['year'] = compustatSC.srcdate.astype('str').str[0:4].astype('int64')

compustatSC = compustatSC.drop(['srcdate'], axis = 1)
compustatSC.rename(columns = {'stic':'TICKER'},inplace = True)

In [ ]:
compustatSC.head()

Merge in the weather information for the suppliers, now that we've added a year to the supply chain date.

This means that we're considering, ie, the weather in 1999 for companies themselves and for companies that they transacted with in 1998.

In [ ]:
hq = pd.read_csv("../../data/allIGData_byQuarter.csv").drop(columns = ['Unnamed: 0'])

hq['year'] = hq.fquarter.str[-4:].astype('int64')

hq = hq.rename(columns={col: 'supplier_' + col 
                        for col in hq.columns if col not in ['company', 'TICKER','fquarter','year']})
hq.head()

In [ ]:
suppliers = compustatSC.merge(hq).drop(columns = {'TICKER','cconm','company'}).rename(columns = {'cgvkey': 'gvkey'})

In [ ]:
suppliers.head()

In [ ]:
compustatHQs = pd.read_csv("../../data/compustatWithHQ_all.csv").\
    drop(columns = {'Unnamed: 0'})

In [ ]:
compustatHQs.head()

In [ ]:
sampleDataset = compustatHQs.merge(suppliers)

In [ ]:
sampleDataset.head()

In [ ]:
sampleDataset.shape

In [ ]:
sampleDataset.columns

In [ ]:
sampleDataset.head()

In [ ]:
sampleDataset.to_csv("../../data/compustat_supplierWeather.csv")

In [ ]:
sampleDataset.shape